In [ ]:
import numpy as np
import pandas as pd
from urllib import request
import json
import altair as alt

conciertos = pd.read_csv('/content/drive/MyDrive/concierto_database_limpia_2.csv', delimiter=';')
conciertos['Fecha'] = pd.to_datetime(conciertos['Fecha'], format='%d-%m-%Y', errors='coerce')
conciertos = conciertos[conciertos['Cancelado'] != 1]
conciertos['Tipo_evento'] = 'Concierto'

# --- LEER Y PREPARAR FESTIVALES ---
festivales = pd.read_csv('/content/drive/MyDrive/Festivales_database_limpia_oficial_gonzalez.csv', delimiter=';')
festivales['Fecha'] = pd.to_datetime(festivales['Fecha'], format='%d-%m-%Y', errors='coerce')
cancelaciones = [
    'Si. La productora aseguró que hubo problemas logísticos imprevistos',
    'Si. Noix Entertainment afirmo en un comunicado que no fue capaz de cumplir con los estandares necesarios',
    'Si. Iguana Producciones comunico que se debio a circunstancias ajenas a Chile',
    'Si. Fue suspendido por un imprevisto que no fue precisado',
    'Si. Se cancelo por baja venta de entradas',
    'Si. Spider Prod declaro que no iban a lograr brindar lo prometido'
]
festivales = festivales[~festivales['Cancelado'].isin(cancelaciones)].copy()
festivales['Tipo_evento'] = 'Festival'


# --- NORMALIZAR CAMPOS CLAVE ---
columnas_necesarias = ['Fecha', 'Artista', 'Festival', 'Concierto', 'Recinto', 'Tipo_evento']
for col in columnas_necesarias:
    if col not in conciertos.columns:
        conciertos[col] = np.nan
    if col not in festivales.columns:
        festivales[col] = np.nan

# --- UNIFICAR EVENTOS ---
eventos = pd.concat([conciertos, festivales], ignore_index=True)

# --- UNIFICAR Y LIMPIAR RECINTOS ---
def unificar_recinto(r):
    if pd.isna(r):
        return r
    r = r.strip().lower()
    if 'parque estadio nacional' in r:
        return 'Parque Estadio Nacional'
    return r.title()

eventos['Recinto'] = eventos['Recinto'].apply(unificar_recinto)

# FECHA EN ESPAÑOL
meses_es = {
    1: 'Enero', 2: 'Febrero', 3: 'Marzo', 4: 'Abril',
    5: 'Mayo', 6: 'Junio', 7: 'Julio', 8: 'Agosto',
    9: 'Septiembre', 10: 'Octubre', 11: 'Noviembre', 12: 'Diciembre'
}
eventos['Fecha_str'] = eventos['Fecha'].dt.day.astype(str) + ' de ' + \
                       eventos['Fecha'].dt.month.map(meses_es) + ' de ' + \
                       eventos['Fecha'].dt.year.astype(str)

#  UNIFICAR NOMBRE DEL EVENTO
eventos['Nombre_evento'] = eventos['Festival'].combine_first(eventos['Concierto'])

# FESTIVALES: AGRUPAR ARTISTAS POR FECHA Y RECINTO
artistas_por_evento = eventos.groupby(['Fecha', 'Recinto'])['Artista'].apply(lambda x: ', '.join(sorted(x.dropna().unique()))).reset_index()
artistas_por_evento.rename(columns={'Artista': 'Artistas_del_dia'}, inplace=True)
eventos = pd.merge(eventos, artistas_por_evento, on=['Fecha', 'Recinto'], how='left')

# ELIMINAR DUPLICADOS FECHA Y RECINTO
eventos = eventos.drop_duplicates(subset=['Fecha', 'Recinto'])

# --- y_random solo para eventos (no días vacíos) ---
np.random.seed(42)
eventos['y_random'] = np.random.uniform(0, 1, size=len(eventos))

# --- FECHA EN TEXTO ESPAÑOL ---
meses_es = {
    1: 'Enero', 2: 'Febrero', 3: 'Marzo', 4: 'Abril',
    5: 'Mayo', 6: 'Junio', 7: 'Julio', 8: 'Agosto',
    9: 'Septiembre', 10: 'Octubre', 11: 'Noviembre', 12: 'Diciembre'
}
eventos['Fecha_str'] = eventos['Fecha'].dt.day.astype(str) + ' de ' + \
                       eventos['Fecha'].dt.month.map(meses_es) + ' de ' + \
                       eventos['Fecha'].dt.year.astype(str)

# --- AGRUPAR ARTISTAS SI HAY VARIOS POR FECHA Y RECINTO ---
eventos['Nombre_evento'] = eventos['Festival'].combine_first(eventos['Concierto'])
artistas_por_evento = eventos.groupby(['Fecha', 'Recinto'])['Artista'].apply(lambda x: ', '.join(sorted(x.dropna().unique()))).reset_index()
eventos = pd.merge(eventos, artistas_por_evento, on=['Fecha', 'Recinto'], how='left')
eventos = eventos.drop_duplicates(subset=['Fecha', 'Recinto'])

# --- LOCALIZACIÓN ALTAR EN ESPAÑOL ---
with request.urlopen('https://raw.githubusercontent.com/d3/d3-format/master/locale/es-MX.json') as f:
    de_format = json.load(f)
with request.urlopen('https://raw.githubusercontent.com/d3/d3-time-format/master/locale/es-MX.json') as f:
    de_time_format = json.load(f)
alt.renderers.set_embed_options(formatLocale=de_format, timeFormatLocale=de_time_format)

total_conciertos = eventos[eventos['Tipo_evento'] == 'Concierto'].shape[0]
total_festivales = eventos[eventos['Tipo_evento'] == 'Festival'].shape[0]
print (f'Eventos por recinto realizados en la Región Metropolitana en 2024\n({total_conciertos} conciertos y {total_festivales} festivales)')


# Crear el gráfico principal
puntos = alt.Chart(eventos).mark_circle(size=60, opacity=0.85).encode(
    x=alt.X('Fecha:T',
            title='Fecha del evento',
            axis=alt.Axis(format='%b', tickCount='month')),
    y=alt.Y('Recinto:N', title='Recinto'),
    color=alt.Color('Tipo_evento:N',
                    scale=alt.Scale(domain=['Concierto', 'Festival'],
                                    range=['#FF005B',
                     '#49007E']),
                    legend=alt.Legend(title='Tipo de evento')),
    tooltip=[
        alt.Tooltip('Tipo_evento:N', title='Tipo'),
        alt.Tooltip('Nombre_evento:N', title='Evento'),
        alt.Tooltip('Recinto:N', title='Recinto'),
        alt.Tooltip('Fecha:T', title='Fecha')
    ]
)

# Crear líneas horizontales por cada recinto
lineas = alt.Chart(eventos).mark_rule(strokeDash=[1,2], color='lightgray').encode(
    y=alt.Y('Recinto:N')
)

# Combinar
chart_con_lineas = (lineas + puntos).properties(
    width=700,
    height=400,
    title='Eventos por recinto realizados en la Región Metropolitana en 2024'
).interactive()

chart_con_lineas

/tmp/ipython-input-1-687960227.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  conciertos['Tipo_evento'] = 'Concierto'


Eventos por recinto realizados en la Región Metropolitana en 2024
(197 conciertos y 41 festivales)


alt.LayerChart(...)